# Test trajectory planning in generated map

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [ ]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
avg_extraction_time = 0
for i in range(num_scans):
    start_time = time.time()
    scans[i] = pc_to_scan(PC_data[i])
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

### Open-loop registration (odometry)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(drone_orientations[0])
t_abs = drone_positions[0,:].copy()
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

R_hats = []
t_hats = []

loop_closure_dist_thresh = 10.0
index_spacing_thresh = 10

r_errors = []

for i in range(1, num_scans):
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    #R_hat = np.array([[0,1,0],[1,0,0],[0,0,1]]) @ R_hat.T @ np.array([[0,1,0],[1,0,0],[0,0,1]])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

    R_hats.append(R_hat)
    t_hats.append(t_hat)

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

Form pose graph

In [ ]:
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
from graphslam.pose.se3 import PoseSE3

from planeslam.geometry.util import R_to_quat

In [ ]:
vertices = []
edges = []

# Add first vertex
p = PoseSE3(traj_est[0], R_to_quat(traj_Rs[:,:,0]))
v = Vertex(0, p)
vertices.append(v)

# For each scan
for i in range(1, num_scans):
    # Add new vertex
    p = PoseSE3(traj_est[i], R_to_quat(traj_Rs[:,:,i]))
    v = Vertex(i, p)
    vertices.append(v)

    # Add odometry edge
    information = np.eye(6)
    estimate = PoseSE3(t_hats[i-1], R_to_quat(R_hats[i-1]))
    estimate.normalize()
    e = EdgeOdometry([i-1, i], information, estimate)
    edges.append(e)

In [ ]:
# Add loop closure edge
R_hat, t_hat = robust_GN_register(scans[59], scans[0])
information = np.eye(6)
estimate = PoseSE3(t_hat, R_to_quat(R_hat))
estimate.normalize()
e = EdgeOdometry([0, 59], information, estimate)
edges.append(e)

In [ ]:
g = Graph(edges, vertices)

In [ ]:
g.optimize()

In [ ]:
positions = np.zeros((num_scans, 3))
for i, v in enumerate(g._vertices):
    positions[i] = v.pose.position

In [ ]:
rotations = np.zeros((3, 3, num_scans))
for i, v in enumerate(g._vertices):
    rotations[:,:,i] = quat_to_R(v.pose.orientation)

In [ ]:
est_traj_trace = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(positions)))
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Generate Map

In [ ]:
# Initialize transformed scans
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans_transformed[i] = deepcopy(scans[i])
    scans_transformed[i].transform(rotations[:,:,i], positions[i])

In [ ]:
merged = scans_transformed[0]

for s in scans_transformed[1:50]:
    merged = merged.merge(s, dist_thresh=7.5)
    merged.reduce_inside(p2p_dist_thresh=5)
    merged.remove_small_planes(area_thresh=1.0)
    merged.fuse_edges(vertex_merge_thresh=2.0)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Trajectory planning

In [ ]:
from planeslam.planning.LPM import LPM

In [ ]:
LPM_file = os.path.join(os.getcwd(),'..', 'data', 'planning_models', 'quadrotor_linear_planning_model.mat')
LPM = LPM(LPM_file)

In [ ]:
k = np.array([[0, 0, 1],
              [0, 0, 1],
              [0, 0, 1]])
positions = LPM.compute_positions(k)

In [ ]:
plane = merged.planes[1]

In [ ]:
fig = go.Figure(data=plane.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
plane.basis

In [ ]:
plane.vertices

In [ ]:
# Check if line segment intersects with plane

line = np.array([[10, -23, 8],
                 [30, -25, 8]])

# Project to plane basis
plane_proj = plane.vertices @ plane.basis
line_proj = line @ plane.basis

# Find intersection point of infinite line with infinite plane
# infinite plane parameterized by (n,d)
# infinite line parameterized by 
line_proj

# Check if intersection point lies within line segment 

In [ ]:
# Check if box intersects with plane (line segment method)

In [ ]:
# Check if box intersects with plane (zonotope method)
# Box zonotope representation
c_box = np.array([0,0,0])[:,None]
G_box = np.eye(3)
box_zono = []

# Plane zonotope representation

In [ ]:
(plane.basis @ line.T).T

In [ ]:
def check_plane_collision(positions, plane):
    c_obs, r_obs = obs
    d_vec = np.linalg.norm(positions - c_obs[:,None], axis=0)
    if any(d_vec <= r_collision + r_obs):
        return False
    else:
        return True

In [ ]:
start = np.array([0.0, 0.0])
goal = np.array([-80.0, -60.0])

